In [1]:
import numpy as np
import os
import scipy.stats as stats
import scipy.io as io
import networkx as nx
import random
import itertools
import glob
from scipy.stats import linregress
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn import svm
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix,roc_curve,roc_auc_score
from xgboost import XGBClassifier
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
import seaborn as sns
from scipy.io import loadmat, savemat
#from fracModel import fracOrdUU
from collections import Counter
from collections import defaultdict
from numpy import arange,array,ones,linalg
from sklearn.linear_model import LinearRegression
import pandas as pd
import powerlaw
import os.path
from os import path
import warnings
warnings.filterwarnings("ignore")
import pickle
import ast

In [2]:
def seq_generator(length):
    s_origin= [random.choice('acgt') for _ in range(length)]
    s_final = ''.join(s_origin)
    return s_final

In [3]:
def savefig(name):
    plt.savefig(name+".svg", bbox_inches = 'tight',dpi=600,format='svg', transparent=True)

In [10]:
##Calculate entropy
def Entropy_MC(s,b,k,beta,gamma,st_in,l):
    s = s[st_in:min(st_in+l,len(s))]
    inv, l, n = k-b, len(s), 4**k
    T=np.zeros((n,n))
    count = [0]*n
    word_list = [''.join(x) for x in itertools.product('acgt', repeat=k)]
    wd = dict(zip(word_list, list(range(len(word_list)))))
    uu = []
    for i in range(k,l-b):
        n1, n2 = wd[s[i-k:i]], wd[s[i-k+b:i+b]]
        T[n1,n2] += 1
        count[n1] += 1
        
    wo_next = [''.join(x) for x in itertools.product('acgt', repeat=b)]
    for i in range(n):
        wo = word_list[i]
        for l in wo_next:
            j = wd[wo[b:]+l]
            T[i,j] = (T[i,j]+beta)/(count[i]+4**b*beta)
    sum_count = np.sum(count)+4**k*gamma
    
    prob = [(count[i]+gamma)/sum_count for i in range(len(count))] 
    n = len(T)
    H = 0
    F = []
    for i in range(n):
        wo = word_list[i]
        for l in wo_next:
            j = wd[wo[b:]+l]
            F.append(T[i,j])
            H-=prob[i]*T[i,j]*np.log2(T[i,j])
    return T,F,H

In [5]:
from collections import defaultdict
def getSeqs(filename):
    seq_list = defaultdict(list)
    mapseq_list = defaultdict(list)
    mapping = {'A': 'a', 'T': 't', 'C': 'c', 'G': 'g','a': 'a', 't': 't', 'c': 'c', 'g': 'g'}
    with open(filename) as f:
        j = -1
        for i, line in enumerate(f):
            if line.startswith('>'):
                j += 1
            else:
                this_line = list(line)
                this_line = list(filter(lambda ch: ch in 'acgtACGT', this_line))
                seq_list[j].extend(this_line)
                mapseq_list[j] = ''.join(list(map(lambda ch: mapping[ch], seq_list[j])))
    return mapseq_list

In [6]:
def stats(s,b,k,beta,gamma,st_in,l):
    F_list = []
    en_list=[]
    for i in range(len(s)):
        F_1,en = Entropy_MC(s[i],b,k,beta,gamma,st_in,l) #Transition Probability vector
        F = F_1#+F_2  #Append TP and FD vector together
        F_list.append(F)
        en_list.append(en)
    return F_list,en_list

In [7]:
def conv_index(b,k,ind):
    word_list = [''.join(x) for x in itertools.product('acgt', repeat=k)]
    wd_next = [''.join(x) for x in itertools.product('acgt', repeat=b)]
    l = 4**b
    rem = ind%l
    st =' -> '+wd_next[rem]
    num = int(ind/l)
    st = word_list[num]+st
    return st
def normalize(x):
        return (x - x.min(0)) / x.ptp(0)

In [7]:
def ML(b,k,V,beta,gamma,classes,st_in,l):
    word_list = [''.join(x) for x in itertools.product('acgt', repeat=k)]
    wd_next = [''.join(x) for x in itertools.product('acgt', repeat=b)]
    X = []
    y = []
    ct = 0
    n = len(V)
    G = [[] for i in range(n)]
    
    for seq in V:
        for s in seq:
            F_list,en_list = stats(s,b,k,beta,gamma,st_in,l)
            G[ct]+=F_list
#         print(len(G[ct]))
        ct+=1
    
    
    min_val = float('inf')
    for i in range(n):
        if min_val > len(G[i]):
            min_val = len(G[i])
   
    
    for i in range(n):
        random.shuffle(G[i])
        X+=[G[i][j] for j in range(len(G[i]))]
        y+=[i for j in range(len(G[i]))]
  
    X_original = [x_i for x_i in X]
    X = np.array(X)
    y = np.array(y)
    X_original = np.array(X_original)
    X = preprocessing.scale(X)
    U_X = list(X[:min_val])
    U_y = list(y[:min_val])
    T_X = list(X[min_val:len(G[0])])
    T_y = list(y[min_val:len(G[0])])
    running_sum = len(G[0])
    for i in range(1,n):
        U_X+=list(X[running_sum:running_sum+min_val])
        U_y+=list(y[running_sum:running_sum+min_val])
        T_X+=list(X[running_sum+min_val:running_sum+len(G[i])])
        T_y+=list(y[running_sum+min_val:running_sum+len(G[i])])
        running_sum+=len(G[i])
    zipped_U = list(zip(U_X,U_y))
    random.shuffle(zipped_U)
    U_X = [x_i for x_i,_ in zipped_U]
    U_y = [y_i for _,y_i in zipped_U]
    zipped_T = list(zip(T_X,T_y))
    random.shuffle(zipped_T)
    T_X = [x_i for x_i,_ in zipped_T]
    T_y = [y_i for _,y_i in zipped_T]
    U_X = np.array(U_X)
    U_y = np.array(U_y)
#     print('Length of X: {}'.format(len(U_y)))
    T_X = np.array(T_X)
    T_y = np.array(T_y)
    return U_X,U_y,X_original,T_X,T_y


In [8]:
def build_model(X,y,X_original,b,k,rr,cutoff,T,y_T,f):    #Building model for feature selection
    kf = KFold(n_splits = 4)
    Acc = []
    Acc_T = []
    sc_prob_T = []
    n = len(rr)
    val_score = [[] for i in range(n)]
    feat_set = set()
    feat_set_ind = set()
    D = {}
    X_orig_mean = np.mean(X_original,axis = 0)
    X_orig_std = np.std(X_original,axis = 0)
#     fig, ax = plt.subplots()
    gg = 0
    mean_fpr = np.linspace(0, 1, 100)
    tprs = []
    aucs = []
    prec = []
    rec = []
    n_dim = 4**(k+b) 
    acc_list = []
#     plt.rcParams.update({'font.size': 14})
    for train, test in kf.split(X):
        X_train = X[train]
        y_train = y[train]
        X_test = X[test]
        y_test = y[test]
        model = XGBClassifier(max_depth=1)
        model.fit(X_train, y_train)
        imp_feat = model.feature_importances_
        imp_indices = np.argsort(imp_feat)
        for im in range(cutoff,0,1):
            transit = conv_index(b,k,imp_indices[im])
            D[transit]  = (round(X_orig_mean[imp_indices[im]],2),round(X_orig_std[imp_indices[im]],4))
            feat_set.add(transit)
            feat_set_ind.add(imp_indices[im])
        #plt.bar(range(len(imp_feat)),imp_feat)
        #plt.show()
        #plt.close()
        imp_feat.sort()
        #feature selection and rebuilding the model using selected features
        thresh = imp_feat[cutoff]
        selection = SelectFromModel(model, threshold=thresh, prefit=True)
        select_X_train = selection.transform(X_train)
        #selection_model = XGBClassifier(max_depth = 1)
        selection_model = LogisticRegression()
        selection_model.fit(select_X_train, y_train)
        select_X_test = selection.transform(X_test)
        scores = selection_model.predict(select_X_test)
        scores_prob = selection_model.predict_proba(select_X_test)
        if len(T) > 0:
            select_T = selection.transform(T)
            scores_T = selection_model.predict(select_T)
            scores_prob_T = selection_model.predict_proba(select_T)
            Acc_T.append(accuracy_score(scores_T,y_T))
            sc_prob_T+=list(scores_prob_T)
        ctr = 0
        for yy in y_test:
            val_score[yy].append(scores_prob[ctr])
            ctr+=1
        ctr = 0
        val_score_dis = [0 for i in range(n)]
        ctx = [0 for i in range(n)]
        for yy in y_test:
            if scores[ctr] == yy:
                val_score_dis[yy]+=1
            ctx[yy]+=1
            ctr+=1
        for yc in range(n):
            val_score_dis[yc] = val_score_dis[yc]/ctx[yc]*100
        acc_list.append(val_score_dis)
        imp_feat_main = selection_model.coef_
        
        Acc.append(accuracy_score(scores,y_test))
        
        gg+=1
    Acc = np.array(Acc)
    val_score = np.array(val_score)
    if len(T)> 0:
        Acc_T = np.array(Acc_T)
        
    acc_list_mean = np.mean(acc_list,axis =0)
    acc_list_std = np.std(acc_list,axis =0)

    for i in range(n):
        probs_mean = np.mean(val_score[i],axis = 0)
        probs_std = np.std(val_score[i],axis = 0)
        
    return np.mean(Acc),np.std(Acc)

In [9]:
# Import pairwise2 module
from Bio import pairwise2

# Import format_alignment method
from Bio.pairwise2 import format_alignment

from tqdm import tqdm
import time

In [18]:
# lengs = [500,1000,2000,4000,8000,16000]
lengs=[leng*1000 for leng in range(1,11)]

for leng in lengs:
    
#     time_list1 = []
#     time_list2 = []
#     time_list3 = []
    time_list4 = []
    
    for i in tqdm(range(20)):
        
        # Generate seq_list A and B
        N = 50 # number of sequances in each pool N=100 done, 50 done, 200 not
        mp = 5
        p = mp/100 # mutation probability
        m = int(leng/1000)+1 # number of mutated bases
        
        L_1, L_2, _, mut_dict,_ = seq_list_generator(N,leng,p,m)

    # Smith-Waterman Algorithm
        end3 = time.time()
        alignments = pairwise2.align.localxx(L_1[25], L_2[25])

        end4 = time.time()
        time_list4.append(N*(end4-end3))
        
        
#     np.save('time1_N_'+str(N)+'_p_'+str(mp)+'_l_'+str(leng), time_list1)
#     np.save('time2_N_'+str(N)+'_p_'+str(mp)+'_l_'+str(leng), time_list2)
#     np.save('time3_N_'+str(N)+'_p_'+str(mp)+'_l_'+str(leng), time_list3)
    np.save('time4_N_'+str(N)+'_p_'+str(mp)+'_l_'+str(leng), time_list4)

100%|██████████| 20/20 [12:33<00:00, 37.66s/it]


In [ ]:
# lengs = [500,1000,2000,4000,8000,16000]
lengs=[leng*1000 for leng in range(1,11)]

for leng in lengs:
    
    time_list1 = []
    time_list2 = []
    time_list3 = []
    time_list4 = []
    
    for i in tqdm(range(50)):
        # Generate seq_list A and B
        N = 50 # number of sequances in each pool N=50 done, 100 done, 200 not
        mp = 10
        p = mp/100 # mutation probability
        m = int(leng/1000)+1 # number of mutated bases

        L_1, L_2, _, mut_dict,_ = seq_list_generator(N,leng,p,m)

        start1 = time.time()

        l = [[1,4]]
        beta = 0.5
        gamma = 0.5
        cutoff = -10

        V = [[L_1],[L_2]]
        b = 1
        k = 4
        st_in = 0
        
        l = 100 # region length
        seq_length = leng # sequence length
        f = open('dummy.txt'.format(l),'w')
        x = int(seq_length/l)+1

        for j in range(1):
            classes = ['L_1','L_2']
            C = V[0]
            D = V[1]
            V_1 = []
            V_1 = [C,D]
            acc_mean = []
            acc_std = []
            for st_in in range(0,seq_length,l):
                X,y,X_original,T,y_T = ML(b,k,V_1,beta,gamma,classes,st_in,l)
                mean,std = build_model(X,y,X_original,b,k,classes,cutoff,T,y_T,f)
                acc_mean.append(mean)
                acc_std.append(std) 
                
        end1 = time.time()
        time_list1.append(end1-start1)

        mt = []
        for i in range(len(acc_mean)):
            if acc_mean[i] > 0.6:
                mt.append(i+1)

        start2 = time.time()
        
#         print(mt)
        for m in mt:
            if m == len(acc_mean):
                X1 = L_1[25][(m-1)*100:]
                Y1 = L_2[25][(m-1)*100:]
                alignments = pairwise2.align.globalxx(X1, Y1)

#                 X2 = L_1[75][(m-1)*100:]
#                 Y2 = L_2[75][(m-1)*100:]
#                 alignments = pairwise2.align.globalxx(X2, Y2)

            else:

                X1 = L_1[25][(m-1)*100:m*100]
                Y1 = L_2[25][(m-1)*100:m*100]
                alignments = pairwise2.align.globalxx(X1, Y1)

#                 X2 = L_1[75][(m-1)*100:m*100]
#                 Y2 = L_2[75][(m-1)*100:m*100]
#                 alignments = pairwise2.align.globalxx(X2, Y2)

        end2 = time.time()

        time_list2.append(start2-end1+N*(end2-start2))


    # Needleman-Wunsch Algorithm

        alignments = pairwise2.align.globalxx(L_1[25], L_2[25])

        end3 = time.time()
        time_list3.append(N*(end3-end2))

    # Smith-Waterman Algorithm
        alignments = pairwise2.align.localxx(L_1[25], L_2[25])

        end4 = time.time()
        time_list4.append(N*(end4-end3))
        
        
    np.save('time1_N_'+str(N)+'_p_'+str(mp)+'_l_'+str(leng), time_list1)
    np.save('time2_N_'+str(N)+'_p_'+str(mp)+'_l_'+str(leng), time_list2)
    np.save('time3_N_'+str(N)+'_p_'+str(mp)+'_l_'+str(leng), time_list3)
    np.save('time3_N_'+str(N)+'_p_'+str(mp)+'_l_'+str(leng), time_list3)

In [ ]:
#N100 p5

lengs=[leng*1000 for leng in range(1,11)]

t1_mean = []
t1_std = []
t2_mean = []
t2_std = []
t3_mean = []
t3_std = []
for leng in lengs:
    t1_list = np.array(np.load('time1_N_100_p_5_l_'+str(leng)+'.npy'))
    t1_mean.append(np.mean(t1_list))
    t1_std.append(np.std(t1_list))
    tm_list = np.array(np.load('time2_N_100_p_5_l_'+str(leng)+'.npy'))
    t2_list = t1_list + tm_list
    t2_mean.append(np.mean(t2_list))
    t2_std.append(np.std(t2_list))
    t3_list = np.array(np.load('time3_N_100_p_5_l_'+str(leng)+'.npy'))
    t3_mean.append(np.mean(t3_list))
    t3_std.append(np.std(t3_list))
    t4_list = np.array(np.load('time4_N_100_p_5_l_'+str(leng)+'.npy'))
    t4_mean.append(np.mean(t4_list))
    t4_std.append(np.std(t4_list))

In [ ]:
plt.xlim([1000,10000])
# plt.ylim([0,750])
plt.rcParams.update({'font.size': 16})
# plt.plot(lengs,t1_mean,linewidth=2)
# plt.fill_between(lengs,np.array(t1_mean)-np.array(t1_std), np.array(t1_mean)+np.array(t1_std), alpha=0.4)
plt.plot(lengs,t2_mean,linewidth=2,label='State Machine')
plt.fill_between(lengs,np.array(t2_mean)-np.array(t2_std), np.array(t2_mean)+np.array(t2_std), alpha=0.4)
plt.plot(lengs,t3_mean,linewidth=2,label='Needleman-Wunsch')
plt.fill_between(lengs,np.array(t3_mean)-np.array(t3_std), np.array(t3_mean)+np.array(t3_std), alpha=0.4)
plt.plot(lengs,t4_mean,linewidth=2,label='Smith-Waterman')
plt.fill_between(lengs,np.array(t4_mean)-np.array(t4_std), np.array(t4_mean)+np.array(t4_std), alpha=0.4)
plt.title('N=100, p=5%')
plt.legend()
plt.ylabel('Time/s')
plt.xlabel('Length/nt')